## Imports

In [1]:
import io
import os.path
from scrapy import Selector
from datetime import datetime
from unicodedata import normalize
from urllib.request import Request, urlopen

In [2]:
url = 'https://fundamentus.com.br/detalhes.php?papel=%s' % 'PETR4'

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

response = urlopen(req, timeout=20).read()
html_dados = response.decode('latin-1')
    
global html_selector
html_selector = Selector( text = html_dados )

In [3]:
len(html_selector.xpath("//table[@class='w728']").extract())

5

In [5]:
len(html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[0::2][0::2])

5

In [16]:
html_selector.xpath("//table[1]").extract() 

['<table id="test1" class="resultado sortable-onload-2-reverse rowstyle-par colstyle-col no-arrow" border="0" cellpadding="0" cellspacing="0">\n<thead>\n<tr>\n<th style="-moz-user-select: none;" class="fd-column-0 sortable-text reverseSort"><a href="#">Papel</a></th>\n<th style="-moz-user-select: none;" class="sortable-text fd-column-1"><a href="#">Nome Comercial</a></th>\n<th style="-moz-user-select: none;" class="sortable-text fd-column-2"><a href="#">Razão Social</a></th>\n</tr>\n</thead>\n<tbody>\n<tr class="">\n<td><a href="detalhes.php?papel=BRDT3">BRDT3</a></td>\n<td>PETROBRAS BR</td>\n<td>PETROBRAS DISTRIBUIDORA S/A</td>\n</tr>\n<tr class="par">\n<td><a href="detalhes.php?papel=CPSL3">CPSL3</a></td>\n<td>COPESUL</td>\n<td>COPESUL - COMPANHIA PETROQUÍMICA DO SUL</td>\n</tr>\n<tr class="">\n<td><a href="detalhes.php?papel=DPPI3">DPPI3</a></td>\n<td>IPIRANGA DISTRIBUIDORA</td>\n<td>DISTRIB PRODUTOS PETROLEO IPIRANGA S.A.</td>\n</tr>\n<tr class="par">\n<td><a href="detalhes.php?pap

In [55]:
info_empresa_label = html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[0::2][0::2]
info_empresa_label = remover_caracteres_especiais(info_empresa_label)
print(info_empresa_label)

[]


## Baixando HTML do site Fundamentus (Web Crawling)

In [2]:
def baixar_html_site_fundamentus(papel):
    print(f"Baixando dados em HTML do '{papel}' no site fundamentus...")
    print("--------------------------------------------")
    url = 'https://fundamentus.com.br/detalhes.php?papel=%s' % papel

    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    response = urlopen(req, timeout=20).read()
    html_dados = response.decode('latin-1')
    
    global html_selector
    html_selector = Selector( text = html_dados )

## Funções para transformar os dados e labels

In [3]:
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

#if __name__ == '__main__':
#    from doctest import testmod
#    testmod()

In [4]:
def remover_caracteres_especiais(empresa_label):
    empresa_label = list(map(lambda x: remover_acentos(x), empresa_label))
    empresa_label = list(map(lambda st: str.replace(st, " ","_"), empresa_label))
    empresa_label = list(map(lambda st: str.replace(st, "(",""), empresa_label))
    empresa_label = list(map(lambda st: str.replace(st, ")",""), empresa_label))
    empresa_label = list(map(lambda st: str.replace(st, ".",""), empresa_label))
    return empresa_label

In [5]:
def validar_data(valor):
    try:
        valor = datetime.strptime(valor, '%d/%m/%Y')
        return valor
    except TypeError:
        return valor

In [6]:
def formatar_tipagem_dados(empresa_dados):
    empresa_dados = list(map(lambda st: str.replace(st, ",","."), empresa_dados))
    
    nova_lista_empresa_dados = []
    for valor in empresa_dados:
        eh_valor_percentual = valor.endswith('%')
        
        if eh_valor_percentual:
            nova_lista_empresa_dados.append(valor)
        elif "/" in valor:
            nova_lista_empresa_dados.append(validar_data(valor))
        elif not eh_valor_percentual:
            try:
                nova_lista_empresa_dados.append(float(valor))
            except ValueError:
                nova_lista_empresa_dados.append(valor)
    
    return nova_lista_empresa_dados

In [7]:
def transformar_lista_em_tuples(empresa_dados):
    empresa_dados_list = []
    empresa_dados_list.append(tuple(empresa_dados))
    return empresa_dados_list

In [8]:
def transformar_listas_em_dicionario(lista_keys, lista_values):
    novo_dicionario = {lista_keys[i]: lista_values[i] for i in range(len(lista_keys))}
    
    return novo_dicionario

## Pegando data última cotação e data último balanço

In [9]:
def extrair_data_ult_cotacao():
    data_ult_cotacao = html_selector.xpath("//table[1]//span[@class='txt']/text() | // table[1]//span[@class='txt']/a/text()").extract()[7]
    data_ult_cotacao = validar_data(data_ult_cotacao)
    return data_ult_cotacao

def extrair_data_ult_balanco():
    data_ult_balanco = html_selector.xpath("//table[2]//span[@class='txt']/text()").extract()[3]
    data_ult_balanco = validar_data(data_ult_balanco)
    return data_ult_balanco

## Verificação se já existe os dados:
* Verificar se o papel já existe nos dados gravados
    * Se não existir, inserir dados da empresa, dados de cotação e dados do balanço
    * Se existir, verificar se a cotação mais atual já existe
    * Se existir, verificar se o balanço mais atual já existe
* Verificando se a cotaçao mais atual já existe
    * Se não existir, inserir dados da cotação atualizado
    * Se existir, não fazer nada
* Verificando se o balanço mais atual já existe
    * Se não existir, inserir dados do balanço atualizado
    * Se existir, não fazer nada

## Conexão com o MongoDB

In [10]:
from pymongo import MongoClient

def conexao_banco_mongodb():    
    cliente = MongoClient('mongodb://localhost:27017/')
    global conexao_banco_mongodb
    conexao_banco_mongodb = cliente['fundamentus']

conexao_banco_mongodb()

### Buscar dados no MongoDB

In [11]:
def buscar_dados_empresa_por_papel(papel):
    colecao = conexao_banco_mongodb.dados_empresa
    dados_empresa = colecao.find_one({"Papel": papel})
    
    return dados_empresa

In [12]:
def buscar_lista_cotacoes_empresa_por_papel(papel):
    colecao = conexao_banco_mongodb.cotacao
    lista_cotacoes = colecao.find({"Papel": papel})
    
    return lista_cotacoes

In [13]:
def buscar_cotacao_empresa_por_papel_data(papel, data_cotacao):
    colecao = conexao_banco_mongodb.cotacao
    cotacao_papel_data = colecao.find_one({"Papel": papel, "Data_ult_cot": data_cotacao})
    
    return cotacao_papel_data

In [14]:
def buscar_lista_balancos_empresa_por_papel(papel):
    colecao = conexao_banco_mongodb.balanco
    lista_balancos = colecao.find({"Papel": papel})
    
    return lista_balancos

In [15]:
def buscar_balanco_empresa_por_papel_data(papel, data_balanco):
    colecao = conexao_banco_mongodb.balanco
    balanco_papel_data = colecao.find_one({"Papel": papel, "Data_ult_cot": data_balanco})
    
    return balanco_papel_data

### Atualizar dados no MongoDB

In [16]:
def atualizar_oscilacao_na_cotacao_mongodb(id_inserido_oscilacoes, id_inserido_cotacao):
    colecao = conexao_banco_mongodb.cotacao
    colecao.update_one({"_id":  id_inserido_cotacao}, {'$set': {'Oscilacao':id_inserido_oscilacoes}})

In [17]:
def atualizar_indicadores_fundamentalistas_na_cotacao_mongodb(id_inserido_indicadores, id_inserido_cotacao):
    colecao = conexao_banco_mongodb.cotacao
    colecao.update_one({"_id":  id_inserido_cotacao}, {'$set': {'Indicador_Fundamentalista': id_inserido_indicadores}})

In [18]:
# Balanço da Empresa
def atualizar_cotacoes_no_balanco_por_id_mongodb(id_inserido_balanco, id_inserido_cotacao):
    colecao = conexao_banco_mongodb.balanco
    colecao.update_one({"_id":  id_inserido_balanco}, {'$push': {'Cotacoes': id_inserido_cotacao }})

In [19]:
def atualizar_cotacoes_no_balanco_por_papel_data_mongodb(papel, data_ult_balanco, id_inserido_cotacao):
    print("*** Atualizando dados de Cotação no Balanço atual da Empresa...")
    colecao = conexao_banco_mongodb.balanco
    colecao.update_one({"Papel":  papel, "Ult_balanco_processado": data_ult_balanco}, {'$push': {'Cotacoes': id_inserido_cotacao }})

### Inserir dados no MongoDB

In [20]:
# Dados da Empresa
def inserir_dados_empresa_mongodb(info_empresa):
    print("** Salvando Informações da Empresa...")
    colecao = conexao_banco_mongodb.dados_empresa
    colecao.insert_one(info_empresa)

In [21]:
# Cotação da Empresa
def inserir_cotacao_empresa_mongodb(cotacao_empresa):
    print("** Salvando dados de Cotação da Empresa...")
    colecao = conexao_banco_mongodb.cotacao
    id_inserido_cotacao = colecao.insert_one(cotacao_empresa).inserted_id
    
    return id_inserido_cotacao

In [22]:
# Oscilações da Empresa
def inserir_oscilacoes_empresa_mongodb(oscilacoes_empresa, id_inserido_cotacao):
    print("** Salvando dados de Oscilações da Empresa...")
    oscilacoes_empresa['Cotacao'] = id_inserido_cotacao
    colecao = conexao_banco_mongodb.oscilacoes
    id_inserido_oscilacoes = colecao.insert_one(oscilacoes_empresa).inserted_id
    
    atualizar_oscilacao_na_cotacao_mongodb(id_inserido_oscilacoes, id_inserido_cotacao)

In [23]:
# Indicadores Fundamentalistas da Empresa
def inserir_indicadores_fundamentalista_empresa_mongodb(indicadores_fundamentalitas_empresa, id_inserido_cotacao):
    print("** Salvando dados de Indicadores Fundamentalistas da Empresa...")
    indicadores_fundamentalitas_empresa['Cotacao'] = id_inserido_cotacao
    colecao = conexao_banco_mongodb.indicadores_fundamentalista
    id_inserido_indicadores = colecao.insert_one(indicadores_fundamentalitas_empresa).inserted_id

    atualizar_indicadores_fundamentalistas_na_cotacao_mongodb(id_inserido_indicadores, id_inserido_cotacao)

In [24]:
def inserir_balanco_empresa_mongodb(balanco_empresa, id_inserido_cotacao):
    print("** Salvando dados de Balanço da Empresa...")
    colecao = conexao_banco_mongodb.balanco
    id_inserido_balanco = colecao.insert_one(balanco_empresa).inserted_id
    
    atualizar_cotacoes_no_balanco_por_id_mongodb(id_inserido_balanco, id_inserido_cotacao)

## Extrair Dados da Empresa e transformar em dicionario

In [25]:
def extrair_info_empresa_label():
    info_empresa_label = html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[0::2][0::2]
    info_empresa_label = remover_caracteres_especiais(info_empresa_label)
    
    return info_empresa_label

In [26]:
def extrair_info_empresa_dados():
    info_empresa_dados = html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[1::2][0::2]
    
    return info_empresa_dados

In [27]:
def criar_dicionario_info_empresa():
    print('* Extraindo Informações da Empresa...')
    info_empresa_label = extrair_info_empresa_label()
    info_empresa_dados = extrair_info_empresa_dados()
    info_empresa = transformar_listas_em_dicionario(info_empresa_label, info_empresa_dados)
    
    return info_empresa

## Extraindo Dados atualizados de Cotação (caso estejam desatualizados)
### Extrair Dados Básicos Cotação e transformar em dicionario

In [28]:
def extrair_cotacao_label():
    cotacao_label = html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[0::2][1::2]
    cotacao_label += html_selector.xpath("//table[2]//span[@class='txt']/text()").extract()[0::2][0::3]
    cotacao_label = remover_caracteres_especiais(cotacao_label)
    cotacao_label.insert(0, 'Papel')    

    return cotacao_label

In [29]:
def extrair_cotacao_dados(papel):
    cotacao_dados = html_selector.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[1::2][1::2]
    cotacao_dados += html_selector.xpath("//table[2]//span[@class='txt']/text()").extract()[1::2][0::3]
    cotacao_dados = formatar_tipagem_dados(cotacao_dados)
    cotacao_dados.insert(0, papel)
    
    return cotacao_dados

In [30]:
def criar_dicionario_cotacao_empresa(papel):
    print('* Extraindo Cotação da Empresa...')
    cotacao_label = extrair_cotacao_label()
    cotacao_dados = extrair_cotacao_dados(papel)
    cotacao_empresa = transformar_listas_em_dicionario(cotacao_label, cotacao_dados)
    
    return cotacao_empresa


### Extrair Dados Oscilação e transformar em dicionario

In [31]:
def extrair_oscilacao_cotacao_label():
    oscilacao_label = html_selector.xpath("//table[3]//td[@class='nivel1']//span/text()").extract()[0]
    lista_oscilacoes_label = html_selector.xpath("//table[3]//td[@class='label w1']//span/text()").extract()
    oscilacoes_label = list(map(lambda label: oscilacao_label + "_" + label, lista_oscilacoes_label))
    oscilacoes_label = remover_caracteres_especiais(oscilacoes_label)
    oscilacoes_label.insert(0, 'Papel')
    
    return oscilacoes_label

In [32]:
def extrair_oscilacao_cotacao_dados(papel):
    oscilacoes_dados = html_selector.xpath("//table[3]//span[@class='oscil']/font/text()").extract()
    oscilacoes_dados.insert(0, papel)
    oscilacoes_dados = formatar_tipagem_dados(oscilacoes_dados)
    
    return oscilacoes_dados

In [33]:
def criar_dicionario_oscilacoes_empresa(papel):
    print('* Extraindo Oscilações de Cotação da Empresa...')
    oscilacoes_label = extrair_oscilacao_cotacao_label()
    oscilacoes_dados = extrair_oscilacao_cotacao_dados(papel)
    oscilacoes_empresa = transformar_listas_em_dicionario(oscilacoes_label, oscilacoes_dados)
    
    return oscilacoes_empresa

### Extrair Dados Indicadores Fundamentalistas e transformar em dicionario

In [34]:
def extrair_indicadores_fundamentalistas_label():
    indicador_fundamentalita_label = html_selector.xpath("//table[3]//td[@class='label w2']//span[@class='txt']/text() | //table[3]//td[@class='label']//span[@class='txt']/text()").extract()
    indicador_fundamentalita_label = remover_caracteres_especiais(indicador_fundamentalita_label)
    indicador_fundamentalita_label.insert(0, 'Papel')
    
    return indicador_fundamentalita_label

In [35]:
def extrair_indicadores_fundamentalistas_dados(papel):
    indicador_fundamentalita_dados = html_selector.xpath("//table[3]//td[@class='data w2']//span[@class='txt']/text() | //table[3]//td[@class='data']//span[@class='txt']/text()").extract()
    indicador_fundamentalita_dados.insert(0, papel)
    indicador_fundamentalita_dados = list(map(lambda dado: dado.replace("\n", ""), indicador_fundamentalita_dados))
    indicador_fundamentalita_dados = formatar_tipagem_dados(indicador_fundamentalita_dados)
    
    return indicador_fundamentalita_dados

In [36]:
def criar_dicionario_indicadores_fundamentalistas_empresa(papel):
    print('* Extraindo Indicadores Fundamentalistas da Empresa...')
    indicador_fundamentalita_label = extrair_indicadores_fundamentalistas_label()
    indicador_fundamentalita_dados = extrair_indicadores_fundamentalistas_dados(papel)
    indicadores_fundamentalistas_empresa = transformar_listas_em_dicionario(indicador_fundamentalita_label, indicador_fundamentalita_dados)

    return indicadores_fundamentalistas_empresa

## Extrair Dados do Balanço e transformar em dicionario

In [37]:
def extrair_balanco_label():
    balanco_label = html_selector.xpath("//table[2]//span[@class='txt']/text()").extract()[2:6][0::2]
    balanco_label += html_selector.xpath("//table[4]//td[@class='label w2']//span[@class='txt']/text() | //table[4]//td[@class='label']//span[@class='txt']/text()").extract()
    dre_12_meses = html_selector.xpath("//table[5]//td[@class='label w2']//span[@class='txt']/text() | //table[5]//td[@class='label']//span[@class='txt']/text()").extract()[0::2]
    balanco_label += list(map(lambda label: label + "_ult._12_meses", dre_12_meses))
    dre_3_meses = html_selector.xpath("//table[5]//td[@class='label w2']//span[@class='txt']/text() | //table[5]//td[@class='label']//span[@class='txt']/text()").extract()[1::2]
    balanco_label += list(map(lambda label: label + "_ult._3_meses", dre_3_meses))
    balanco_label = remover_caracteres_especiais(balanco_label)
    balanco_label.insert(0, 'Papel')
    
    return balanco_label

In [38]:
def extrair_balanco_dados(papel):
    balanco_dados = html_selector.xpath("//table[2]//span[@class='txt']/text()").extract()[2:6][1::2]
    balanco_dados += html_selector.xpath("//table[4]//td[@class='data w3']//span[@class='txt']/text() | //table[4]//td[@class='data']//span[@class='txt']/text()").extract()
    balanco_dados += html_selector.xpath("//table[5]//td[@class='data w3']//span[@class='txt']/text() | //table[5]//td[@class='data']//span[@class='txt']/text()").extract()[0::2]
    balanco_dados += html_selector.xpath("//table[5]//td[@class='data w3']//span[@class='txt']/text() | //table[5]//td[@class='data']//span[@class='txt']/text()").extract()[1::2]
    balanco_dados.insert(0, papel)
    balanco_dados = formatar_tipagem_dados(balanco_dados)
    
    return balanco_dados

In [39]:
def criar_dicionario_balanco_empresa(papel):
    print('* Extraindo dados do Balanço da Empresa...')
    balanco_label = extrair_balanco_label()
    balanco_dados = extrair_balanco_dados(papel)
    balanco_empresa = transformar_listas_em_dicionario(balanco_label, balanco_dados)
    
    return balanco_empresa

## Adicionar Dados a serem inseridos no MongoDB

In [40]:
def adicionar_info_empresa():
    info_empresa = criar_dicionario_info_empresa()
    inserir_dados_empresa_mongodb(info_empresa)

In [41]:
def adicionar_cotacao_empresa(papel):
    cotacao_empresa = criar_dicionario_cotacao_empresa(papel)    
    id_inserido_cotacao = inserir_cotacao_empresa_mongodb(cotacao_empresa)
        
    oscilacoes_empresa = criar_dicionario_oscilacoes_empresa(papel)
    inserir_oscilacoes_empresa_mongodb(oscilacoes_empresa, id_inserido_cotacao)
        
    indicadores_fundamentalistas_empresa = criar_dicionario_indicadores_fundamentalistas_empresa(papel)
    inserir_indicadores_fundamentalista_empresa_mongodb(indicadores_fundamentalistas_empresa, id_inserido_cotacao)
    
    return id_inserido_cotacao

In [42]:
def adicionar_balanco_empresa(papel, id_inserido_cotacao):
    balanco_empresa = criar_dicionario_balanco_empresa(papel)
    inserir_balanco_empresa_mongodb(balanco_empresa, id_inserido_cotacao)

### Verificação se dados já existem no MongoDB

In [43]:
def dados_empresa_nao_existem(papel):
    dados_empresa = buscar_dados_empresa_por_papel(papel)
    if dados_empresa is None:
        return True
    else:
        return False 
    

def ultima_cotacao_nao_existem(papel, data_ult_cotacao):
    ultima_cotacao = buscar_cotacao_empresa_por_papel_data(papel, data_ult_cotacao)
    if ultima_cotacao is None:
        return True
    else:
        return False
    

def ultimo_balanco_nao_existem(papel, data_ult_balanco):
    ultimo_balanco = buscar_balanco_empresa_por_papel_data(papel, data_ult_balanco)
    if ultimo_balanco is None:
        return True
    else:
        return False

## Método principal (main)

In [44]:
def metodo_main():
    
    print('Iniciando Web Scraping:')
    print("--------------------------------------------")
    papel = input("Digite o código do papel: ").upper()
    
    baixar_html_site_fundamentus(papel)
    data_ult_cotacao = extrair_data_ult_cotacao()
    data_ult_balanco = extrair_data_ult_balanco()
    
    # ADICIONAR DADOS DA EMPRESA
    if dados_empresa_nao_existem(papel):
        print(f"Dados da empresa '{papel}' ainda não existem:")
        adicionar_info_empresa()
        id_inserido_cotacao = adicionar_cotacao_empresa(papel)
        adicionar_balanco_empresa(papel, id_inserido_cotacao)

    # ADICIONAR DADOS DA ÚLTIMA COTAÇÃO
    # ADICIONAR DADOS DA ÚLTIMA OSCILAÇÃO
    # ADICIONAR DADOS DOS ÚLTIMOS INDICADORES FUNDAMENTALISTAS
    elif ultima_cotacao_nao_existem(papel, data_ult_cotacao):
        print(f"Dados da empresa '{papel}' já existem:")
        print('* Atualizando Dados de Cotação....')
        id_inserido_cotacao = adicionar_cotacao_empresa(papel)
        
        if ultimo_balanco_nao_existem(papel, data_ult_balanco):
            print("--------------------------------------------")
            print('* Adicionando Balanço atualizado da empresa....')
            adicionar_balanco_empresa(papel, id_inserido_cotacao)
        
        else:
            print("--------------------------------------------")
            print('Dados atuais do Balanço da empresa já existem:')
            print('* Adicionando Balanço atualizado da empresa....')
            atualizar_cotacoes_no_balanco_por_papel_data_mongodb(papel, data_ult_balanco, id_inserido_cotacao)
    else:
        print("--------------------------------------------")
        print('Todos os dados já estão atualizados!')

In [45]:
metodo_main()

Iniciando Web Scraping:
--------------------------------------------


Digite o código do papel:  GGBR4


Baixando dados em HTML do 'GGBR4' no site fundamentus...
--------------------------------------------
Dados da empresa 'GGBR4' ainda não existem:
* Extraindo Informações da Empresa...
** Salvando Informações da Empresa...
* Extraindo Cotação da Empresa...
** Salvando dados de Cotação da Empresa...
* Extraindo Oscilações de Cotação da Empresa...
** Salvando dados de Oscilações da Empresa...
* Extraindo Indicadores Fundamentalistas da Empresa...
** Salvando dados de Indicadores Fundamentalistas da Empresa...
* Extraindo dados do Balanço da Empresa...
** Salvando dados de Balanço da Empresa...


## Buscando dados no MongoDB

In [ ]:
def buscar_balanco_por_papel(papel):
    colecao = conexao_banco_mongodb.balanco
    resultado = colecao.find_one({ "Papel": papel})
    
    return resultado

resultado = buscar_balanco_por_papel('PETR4')
resultado

In [ ]:
def buscar_cotacao_e_dados_oscilacao(papel):
    pipeline = [
        {"$match" : { "Papel" : papel }},
        {"$lookup" : { "from": "oscilacoes",
                       "localField": "Papel",
                       "foreignField": "Papel",
                       "as": "Oscilacao_Papel" }},
        {"$unset": ["Oscilacao_Papel._id", "Oscilacao_Papel.Papel", "Oscilacao_Papel.Cotacao"]} # Ocultando campos no retorno
    ]
    colecao = conexao_banco_mongodb.cotacao
    resultado = list(colecao.aggregate(pipeline))
    
    return resultado

resultado = buscar_cotacao_e_dados_oscilacao("PETR4")
resultado

In [ ]:
resultado[0]['Oscilacao_Papel']

### Testando inclusão de item na lista Cotacoes na coleção Balanço

In [98]:
resultado = buscar_balanco_por_papel("PETR4")
id_balanco = resultado['_id']

atualizar_lista_cotacoes_no_balanco(id_balanco)

NameError: name 'buscar_balanco_por_papel' is not defined

# --------------------------------------------------------------------------------------------

## Verificando se o arquivo já existe

In [37]:
#path = '/home/hugo/Documents/Repositorios_GitHub/Projetos/02-StockData/Web_Scraping/Fundamentus_Web/'
#file_name = '%s_fundamentus_%s.txt' % (papel, str(data_ult_balanco_new_html))

#if os.path.isfile(path+file_name):
#    print('Arquivo já existe com balanço mais atual!')
#else:
#    print('Arquivo com balanço atual ainda não existe!')

## Salvando conteúdo do HTML (em bytes) em um arquivo local como txt

In [38]:
#file_name = '%s_fundamentus_%s.txt' % (papel, data_ult_balanco_new_html)

#with io.open(path+file_name, 'w', encoding='utf-8') as f:
#    f.write(new_html)

## Lendo conteúdo do arquivo Local e armazenando em uma variável

In [39]:
#with io.open(path+file_name, 'r', encoding='utf8') as f:
#    html = f.read()

# -----------------------------------------------------------------------------------------

In [53]:
# ADICIONA ITEM A UMA LISTA

#colecao = banco.cotacao
#colecao.update_one({"_id":  id_inserido_cotacao}, {'$push': {'oscilacoes': id_inserido_cotacao }})

In [59]:
#colecao = banco.balanco

#colecao.update_one({"_id":  id_inserido_balanco}, {'$set': {'Indicador_Fundamentalista': id_inserido_indicadores}})

# -----------------------------------------------------------------------------------------

# Acesso ao HDFS server (Write and Read)
## Criando uma Spark Session

In [40]:
#!pip3 install pyspark

In [41]:
from pyspark.sql import SparkSession

def criar_spark_session():
    global sparkSession
    sparkSession = SparkSession.builder.appName("write-read-fundamentus-hdfs").getOrCreate()
criar_spark_session()

## Write Spark DataFrame dos Dados da Empresa

### Necessário configurar a versão do Python utilizada pelo Linux como a 3.6.
* sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.4 1
* sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.6 2
* sudo update-alternatives --config python
* sudo update-alternatives  --set python /usr/bin/python3.6

In [ ]:
def criar_spark_dataframe(empresa_label, empresa_dados):
    dados_tuple = transformar_lista_em_tuples(empresa_dados)
    empresa_df = sparkSession.createDataFrame(empresa_label, empresa_dados)
    
    return empresa_df

In [ ]:
# Salvar Spark DataFrame no HDFS
def salvar_spark_dataframe_hdfs(url_hdfs, empresa_df):
    empresa_df.write.save(url_hdfs, format='parquet', mode='append')


### URLs dos dados no HDFS

In [ ]:
def retorna_url_hdfs_dados_empresa():
    url_hdfs = 'hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_empresa'
    
    return url_hdfs

In [ ]:
def retorna_url_hdfs_cotacao_empresa():
    url_hdfs = 'hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_cotacao'
    
    return url_hdfs

In [ ]:
def retorna_url_hdfs_oscilacoes_empresa():
    url_hdfs = 'hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_oscilacoes'
    
    return url_hdfs

In [ ]:
def retorna_url_hdfs_indicacores_fundamentalistas_empresa():
    url_hdfs = 'hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_indicador_fundamentalista'
    
    return url_hdfs

In [ ]:
def retorna_url_hdfs_balanco_empresa():
    url_hdfs = 'hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_balanco'
    
    return url_hdfs

### Informações da Empresa

In [ ]:
# Salvar dados da empresa no HDFS
def salvar_fundamentus_dados_empresa():
    url_hdfs = retorna_url_hdfs_dados_empresa()
    
    dataframe_label = extrair_info_empresa_label()
    dataframe_dados = extrair_info_empresa_dados()
    
    empresa_df = criar_spark_dataframe(dataframe_label, dataframe_dados)
    
    salvar_spark_dataframe_hdfs(url_hdfs, empresa_df)
    
salvar_fundamentus_dados_empresa()

In [ ]:
# Ler dados da empresa no HDFS
def ler_fundamentus_dados_empresa():
    url_hdfs = retorna_url_hdfs_dados_empresa()
    empresa_df = sparkSession.read.format('parquet').load(url_hdfs)
    
    return empresa_df

In [ ]:
empresa_df = ler_fundamentus_dados_empresa()
empresa_df.show()

In [ ]:
empresa_df.toPandas()

### Cotações da Empresa

In [ ]:
# Salvar cotação da ação da empresa no HDFS
def salvar_fundamentus_cotacao_empresa():
    url_hdfs = retorna_url_hdfs_cotacao_empresa()
    
    dataframe_label = extrair_cotacao_label()
    dataframe_dados = extrair_cotacao_dados()
    
    empresa_df = criar_spark_dataframe(dataframe_label, dataframe_dados)
    
    salvar_spark_dataframe_hdfs(url_hdfs, empresa_df)
    
salvar_fundamentus_cotacao_empresa()

In [ ]:
# Ler cotação da ação da empresa no HDFS
def ler_fundamentus_cotacao_empresa():
    url_hdfs = retorna_url_hdfs_cotacao_empresa()
    empresa_df = sparkSession.read.format('parquet').load(url_hdfs)
    
    return empresa_df

In [ ]:
empresa_df = ler_fundamentus_cotacao_empresa()
empresa_df.show()

In [ ]:
empresa_df.toPandas()

### Oscilações Empresa

In [ ]:
# Salvar oscilações do papel da empresa no HDFS
def salvar_fundamentus_oscilacoes_empresa():
    url_hdfs = retorna_url_hdfs_oscilacoes_empresa()
    
    dataframe_label = extrair_oscilacao_cotacao_label()
    dataframe_dados = extrair_oscilacao_cotacao_dados()
    
    empresa_df = criar_spark_dataframe(dataframe_label, dataframe_dados)
    
    salvar_spark_dataframe_hdfs(url_hdfs, empresa_df)
    
salvar_fundamentus_oscilacoes_empresa()

In [ ]:
# Ler oscilações do papel da empresa no HDFS
def ler_fundamentus_oscilacoes_empresa():
    url_hdfs = retorna_url_hdfs_oscilacoes_empresa()
    empresa_df = sparkSession.read.format('parquet').load(url_hdfs)
    
    return empresa_df

In [ ]:
empresa_df = ler_fundamentus_oscilacoes_empresa()
empresa_df.show()

In [ ]:
empresa_df.toPandas()

### Indicadores Fundamentalistas

In [ ]:
# Salvar indicadores fundamentalistas da empresa no HDFS
def salvar_fundamentus_indicacores_fundamentalistas_empresa():
    url_hdfs = retorna_url_hdfs_indicacores_fundamentalistas_empresa()
    
    dataframe_label = extrair_indicacores_fundamentalistas_label()
    dataframe_dados = extrair_indicacores_fundamentalistas_dados()
    
    empresa_df = criar_spark_dataframe(dataframe_label, dataframe_dados)
    
    salvar_spark_dataframe_hdfs(url_hdfs, empresa_df)
    
salvar_fundamentus_indicacores_fundamentalistas_empresa()

In [ ]:
# Ler indicadores fundamentalistas da empresa no HDFS
def ler_fundamentus_indicacores_fundamentalistas_empresa():
    url_hdfs = retorna_url_hdfs_indicacores_fundamentalistas_empresa()
    empresa_df = sparkSession.read.format('parquet').load(url_hdfs)
    
    return empresa_df

In [ ]:
empresa_df = ler_fundamentus_indicacores_fundamentalistas_empresa()
empresa_df.show()

In [ ]:
empresa_df.toPandas()

### Balanço Empresa

In [ ]:
# Salvar balanço da empresa no HDFS
def salvar_fundamentus_balanco_empresa():
    url_hdfs = retorna_url_hdfs_balanco_empresa()
    
    dataframe_label = extrair_balanco_label()
    dataframe_dados = extrair_balanco_dados()
    
    empresa_df = criar_spark_dataframe(dataframe_label, dataframe_dados)
    
    salvar_spark_dataframe_hdfs(url_hdfs, empresa_df)
    
salvar_fundamentus_balanco_empresa()

In [ ]:
# Ler balanço da empresa no HDFS
def ler_fundamentus_balanco_empresa():
    url_hdfs = retorna_url_hdfs_balanco_empresa()
    empresa_df = sparkSession.read.format('parquet').load(url_hdfs)
    
    return empresa_df

In [ ]:
empresa_df = ler_fundamentus_balanco_empresa()
empresa_df.show()

In [ ]:
empresa_df.toPandas()

In [2]:
#sparkSession.createDataFrame(info_empresa_dados).collect()

In [3]:
#sparkSession.createDataFrame(info_empresa_dados, info_empresa_label).collect()

In [4]:
#df.show()

## Write Spark DataFrame dos Dados de Cotação

In [ ]:
cotacao_label.insert(0, 'Papel')
cotacao_dados.insert(0, papel)
cotacao_dados = transformar_lista_em_tuples(cotacao_dados)
cotacao_dados

In [51]:
df = sparkSession.createDataFrame(cotacao_dados, cotacao_label)
df

DataFrame[Papel: string, Cotacao: string, Data_ult_cot: string, Min_52_sem: string, Max_52_sem: string, Vol_$_med_2m: string, Valor_de_mercado: string, Nro._Acoes: string]

In [52]:
df.write.save('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_cotacao', format='parquet', mode='append')

In [53]:
df_load = sparkSession.read.format('parquet').load('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_cotacao')

In [54]:
df_load.show()

+-----+-------+------------+----------+----------+-------------+----------------+--------------+
|Papel|Cotacao|Data_ult_cot|Min_52_sem|Max_52_sem| Vol_$_med_2m|Valor_de_mercado|    Nro._Acoes|
+-----+-------+------------+----------+----------+-------------+----------------+--------------+
|VALE3| 101,60|  02/03/2021|     32,82|    102,32|3.111.610.000| 536.903.000.000| 5.284.470.000|
|PETR4|  21,99|  02/03/2021|     11,29|     31,12|2.600.720.000| 286.848.000.000|13.044.500.000|
|ITUB4|  25,77|  02/03/2021|     20,24|     32,58|1.217.880.000| 252.653.000.000| 9.804.140.000|
|GGBR4|  26,67|  01/03/2021|      8,54|     29,54|  419.645.000|  45.863.300.000| 1.719.660.000|
+-----+-------+------------+----------+----------+-------------+----------------+--------------+



In [55]:
df_load.toPandas()

,Papel,Cotacao,Data_ult_cot,Min_52_sem,Max_52_sem,Vol_$_med_2m,Valor_de_mercado,Nro._Acoes
0,VALE3,"101,60",02/03/2021,"32,82","102,32",3.111.610.000,536.903.000.000,5.284.470.000
1,PETR4,"21,99",02/03/2021,"11,29","31,12",2.600.720.000,286.848.000.000,13.044.500.000
2,ITUB4,"25,77",02/03/2021,"20,24","32,58",1.217.880.000,252.653.000.000,9.804.140.000
3,GGBR4,"26,67",01/03/2021,"8,54","29,54",419.645.000,45.863.300.000,1.719.660.000


## Write Spark DataFrame dos Dados de Oscilações

In [ ]:
oscilacoes_label.insert(0, 'Papel')
oscilacoes_dados.insert(0, papel)
oscilacoes_dados = transformar_lista_em_tuples(oscilacoes_dados)
oscilacoes_dados

In [56]:
df = sparkSession.createDataFrame(oscilacoes_dados, oscilacoes_label)
df

DataFrame[Papel: string, Oscilacoes_Dia: string, Oscilacoes_Mes: string, Oscilacoes_30_dias: string, Oscilacoes_12_meses: string, Oscilacoes_2021: string, Oscilacoes_2020: string, Oscilacoes_2019: string, Oscilacoes_2018: string, Oscilacoes_2017: string, Oscilacoes_2016: string]

In [57]:
df.write.save('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_oscilacoes', format='parquet', mode='append')

In [58]:
df_load = sparkSession.read.format('parquet').load('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_oscilacoes')

In [59]:
df_load.show()

+-----+--------------+--------------+------------------+-------------------+---------------+---------------+---------------+---------------+---------------+---------------+
|Papel|Oscilacoes_Dia|Oscilacoes_Mes|Oscilacoes_30_dias|Oscilacoes_12_meses|Oscilacoes_2021|Oscilacoes_2020|Oscilacoes_2019|Oscilacoes_2018|Oscilacoes_2017|Oscilacoes_2016|
+-----+--------------+--------------+------------------+-------------------+---------------+---------------+---------------+---------------+---------------+---------------+
|PETR4|        -0,05%|        -1,12%|           -23,27%|            -17,11%|        -22,41%|         -6,10%|         36,87%|         45,91%|          8,29%|        116,23%|
|ITUB4|         4,04%|         0,93%|            -8,90%|            -18,04%|        -17,91%|        -11,75%|         11,98%|         32,41%|         29,64%|         56,02%|
|VALE3|         3,07%|         7,49%|            15,90%|            127,71%|         16,18%|         70,47%|          6,85%|         31

In [60]:
df_load.toPandas()

,Papel,Oscilacoes_Dia,Oscilacoes_Mes,Oscilacoes_30_dias,Oscilacoes_12_meses,Oscilacoes_2021,Oscilacoes_2020,Oscilacoes_2019,Oscilacoes_2018,Oscilacoes_2017,Oscilacoes_2016
0,PETR4,"-0,05%","-1,12%","-23,27%","-17,11%","-22,41%","-6,10%","36,87%","45,91%","8,29%","116,23%"
1,ITUB4,"4,04%","0,93%","-8,90%","-18,04%","-17,91%","-11,75%","11,98%","32,41%","29,64%","56,02%"
2,VALE3,"3,07%","7,49%","15,90%","127,71%","16,18%","70,47%","6,85%","31,11%","62,56%","103,66%"
3,GGBR4,"3,09%","3,09%","16,11%","55,83%","9,08%","23,86%","37,65%","22,31%","15,23%","138,66%"


## Write Spark DataFrame dos Dados de Indicacores Fundamentalistas

In [ ]:
indicador_fundamentalita_label.insert(0, "Papel")
indicador_fundamentalita_dados.insert(0, papel)
indicador_fundamentalita_dados = transformar_lista_em_tuples(indicador_fundamentalita_dados)
indicador_fundamentalita_dados

In [61]:
df = sparkSession.createDataFrame(indicador_fundamentalita_dados, indicador_fundamentalita_label)
df

DataFrame[Papel: string, P/L: string, LPA: string, P/VP: string, VPA: string, P/EBIT: string, Marg._Bruta: string, PSR: string, Marg._EBIT: string, P/Ativos: string, Marg._Liquida: string, P/Cap._Giro: string, EBIT_/_Ativo: string, P/Ativ_Circ_Liq: string, ROIC: string, Div._Yield: string, ROE: string, EV_/_EBITDA: string, Liquidez_Corr: string, EV_/_EBIT: string, Div_Br/_Patrim: string, Cres._Rec_5a: string, Giro_Ativos: string]

In [62]:
df.write.save('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_indicador_fundamentalista', format='parquet', mode='append')

In [63]:
df_load = sparkSession.read.format('parquet').load('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_indicador_fundamentalista')

In [64]:
df_load.show()

+-----+-----+----+----+-----+------+-----------+----+----------+--------+-------------+-----------+------------+---------------+-----+----------+-----+-----------+-------------+---------+--------------+------------+-----------+
|Papel|  P/L| LPA|P/VP|  VPA|P/EBIT|Marg._Bruta| PSR|Marg._EBIT|P/Ativos|Marg._Liquida|P/Cap._Giro|EBIT_/_Ativo|P/Ativ_Circ_Liq| ROIC|Div._Yield|  ROE|EV_/_EBITDA|Liquidez_Corr|EV_/_EBIT|Div_Br/_Patrim|Cres._Rec_5a|Giro_Ativos|
+-----+-----+----+----+-----+------+-----------+----+----------+--------+-------------+-----------+------------+---------------+-----+----------+-----+-----------+-------------+---------+--------------+------------+-----------+
|VALE3|20,10|5,05|2,89|35,16|  5,01|      52,7%|2,57|     51,4%|    1,12|        11,9%|      10,53|       22,4%|          -3,15|27,4%|      2,4%|14,4%|       4,40|         1,67|     5,09|          0,42|       20,9%|       0,44|
|PETR4|40,36|0,54|0,93|23,64|  3,07|      45,6%|1,05|     34,3%|    0,29|         2,3%| 

In [65]:
df_load.toPandas()

,Papel,P/L,LPA,P/VP,VPA,P/EBIT,Marg._Bruta,PSR,Marg._EBIT,P/Ativos,...,P/Ativ_Circ_Liq,ROIC,Div._Yield,ROE,EV_/_EBITDA,Liquidez_Corr,EV_/_EBIT,Div_Br/_Patrim,Cres._Rec_5a,Giro_Ativos
0,VALE3,"20,10","5,05","2,89","35,16","5,01","52,7%","2,57","51,4%","1,12",...,"-3,15","27,4%","2,4%","14,4%","4,40","1,67","5,09","0,42","20,9%","0,44"
1,PETR4,"40,36","0,54","0,93","23,64","3,07","45,6%","1,05","34,3%","0,29",...,"-0,54","10,5%","0,0%","2,3%","3,92","1,04","6,58","1,27","-0,1%","0,28"
2,GGBR4,"19,39","1,38","1,49","17,95","10,42","13,5%","1,05","10,0%","0,73",...,"-5,32","8,8%","1,1%","7,7%","8,14","2,04","12,66","0,57","3,8%","0,69"
3,ITUB4,"13,32","1,93","1,85","13,94",-,-,-,-,-,...,-,-,"1,9%","13,9%",-,-,-,-,"-30,0%",-


## Write Spark DataFrame dos Dados do Balanço

In [ ]:
balanco_label.insert(0, "Papel")
balanco_dados.insert(0, papel)
balanco_dados = transformar_lista_em_tuples(balanco_dados)
balanco_dados

In [66]:
df = sparkSession.createDataFrame(balanco_dados, balanco_label)
df

DataFrame[Papel: string, Ult_balanco_processado: string, Valor_da_firma: string, Ativo: string, Div._Bruta: string, Disponibilidades: string, Div._Liquida: string, Ativo_Circulante: string, Patrim._Liq: string, Receita_Liquida_ult._12_meses: string, EBIT_ult._12_meses: string, Lucro_Liquido_ult._12_meses: string, Receita_Liquida_ult._3_meses: string, EBIT_ult._3_meses: string, Lucro_Liquido_ult._3_meses: string]

In [67]:
df.write.save('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_balanco', format='parquet', mode='append')

In [68]:
df_load = sparkSession.read.format('parquet').load('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_balanco')

In [69]:
df_load.show()

+-----+----------------------+---------------+---------------+---------------+----------------+---------------+----------------+---------------+-----------------------------+------------------+---------------------------+----------------------------+-----------------+--------------------------+
|Papel|Ult_balanco_processado| Valor_da_firma|          Ativo|     Div._Bruta|Disponibilidades|   Div._Liquida|Ativo_Circulante|    Patrim._Liq|Receita_Liquida_ult._12_meses|EBIT_ult._12_meses|Lucro_Liquido_ult._12_meses|Receita_Liquida_ult._3_meses|EBIT_ult._3_meses|Lucro_Liquido_ult._3_meses|
+-----+----------------------+---------------+---------------+---------------+----------------+---------------+----------------+---------------+-----------------------------+------------------+---------------------------+----------------------------+-----------------+--------------------------+
|PETR4|            31/12/2020|615.116.000.000|987.419.000.000|392.548.000.000|  64.280.000.000|328.268.000.000| 

In [70]:
df_load.toPandas()

,Papel,Ult_balanco_processado,Valor_da_firma,Ativo,Div._Bruta,Disponibilidades,Div._Liquida,Ativo_Circulante,Patrim._Liq,Receita_Liquida_ult._12_meses,EBIT_ult._12_meses,Lucro_Liquido_ult._12_meses,Receita_Liquida_ult._3_meses,EBIT_ult._3_meses,Lucro_Liquido_ult._3_meses
0,PETR4,31/12/2020,615.116.000.000,987.419.000.000,392.548.000.000,64.280.000.000,328.268.000.000,142.323.000.000,308.410.000.000,272.069.000.000,93.417.000.000,7.108.000.000,74.972.000.000,33.838.000.000,59.890.000.000
1,VALE3,31/12/2020,544.906.000.000,478.129.000.000,78.089.100.000,70.085.600.000,8.003.530.000,126.805.000.000,185.785.000.000,208.529.000.000,107.104.000.000,26.712.700.000,78.938.200.000,47.152.400.000,4.824.830.000
2,GGBR4,31/12/2020,55.720.300.000,63.123.000.000,17.515.400.000,7.658.350.000,9.857.000.000,23.409.500.000,30.860.300.000,43.814.700.000,4.400.170.000,2.365.760.000,13.620.200.000,2.176.850.000,1.048.330.000
3,ITUB4,31/12/2020,-,210.399.000.000,None,None,None,None,136.699.000.000,None,None,18.961.000.000,None,None,7.636.000.000
